In [1]:
import pandas as pd
import numpy as np
import os
os.chdir(r"D:\Research\UBI车险\data\demo_data")

In [2]:
# import data
x_col_names = ["Distance","Average_speed","acceleration_events","sudden_starts","abrupt_lane_changes","intense_brakes","sudden_stops","abrupt_steerings"]
def import_data(i):
    file_name = "fold_{}_1-10.xlsx".format(i)
    x_train = pd.DataFrame(columns = x_col_names)
    x_test = pd.DataFrame(columns = x_col_names)
    y_train = pd.DataFrame(columns = ["score"])
    y_test = pd.DataFrame(columns = ["score"])

    x_train_part = pd.read_excel(file_name,header = None, names = x_col_names,sheet_name = "xtrain")
    x_test_part = pd.read_excel(file_name,header = None, names = x_col_names,sheet_name = "xtest")
    x_train = pd.concat([x_train,x_train_part],axis = 0)
    x_test = pd.concat([x_test,x_test_part],axis = 0)

    y_train_part = pd.read_excel(file_name,header = None, names = ["score"],sheet_name = "ytrain")
    y_test_part = pd.read_excel(file_name,header = None, names = ["score"],sheet_name = "ytest")
    y_train = pd.concat([y_train,y_train_part],axis = 0)
    y_test = pd.concat([y_test,y_test_part],axis = 0)

    df_train = pd.concat([x_train,y_train],axis = 1)
    df_test = pd.concat([x_test,y_test],axis = 1)
    df_train = df_train.drop_duplicates(subset = x_col_names)
    df_test = df_test.drop_duplicates(subset = x_col_names)

    for col in x_col_names + ["score"]:
        df_train[col] = df_train[col].astype(int)
        df_test[col] = df_test[col].astype(int)
    return df_train,df_test

In [15]:
df_train,df_test = import_data(1)
df_train.head(5)

,Distance,Average_speed,acceleration_events,sudden_starts,abrupt_lane_changes,intense_brakes,sudden_stops,abrupt_steerings,score
0,5,8,3,6,3,3,4,5,4
1,9,9,5,5,3,0,0,6,6
2,3,3,2,2,3,2,2,0,7
3,7,4,3,3,3,5,2,4,4
4,9,10,6,3,8,5,0,4,4


In [3]:
# define varaibles

$y$ = score

$x_1$ = Distance, 

$x_2$ = Average speed, 

$x_3$ = # of acceleration events, 

$x_4$ = # of sudden starts, 

$x_5$ = # of abrupt lane changes, 

$x_6$ = # intence brakes

$x_7$ = # of sudden stops, 

$x_8$ = # of abrupt steering

In [ ]:
# 5 Machine Learning Methods

Driver Safety Index(Saiprasert et al., 2014)

Genetic Programming(López et al., 2018)

Bayesian Ridge Regression(Eran et al., 2012)

Random Forest(Breiman, 2001)

neural network(Haykin, 2003)


In [ ]:
# Model Evaluation

$$
RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^n (\hat{y} - y)^2}
$$

In [4]:
RMSE_model_list = []

In [5]:
# Driver Safety Index(Saiprasert et al., 2014)

$$
y = 1 - \frac{1}{x_1} ( -1.0580 x_2 +  1.3695 x_3 +  2.3714 x_4 +  1.2127 x_5 +  1.7082 x_6 +  0.4527 x_7 +  1.2374 x_8 )
$$

In [6]:
from sklearn.linear_model import LinearRegression
RMSE_list = []
for i in range(1,11):
    df_train,df_test = import_data(i)
    lm_model = LinearRegression(fit_intercept = False)
    x_col_names_lm = ["Average_speed","acceleration_events","sudden_starts","abrupt_lane_changes","intense_brakes","sudden_stops","abrupt_steerings"]
    x_train_lm = np.array(df_train[x_col_names_lm]) / np.array(df_train["Distance"]).reshape(len(df_train),1)
    x_test_lm = np.array(df_test[x_col_names_lm]) / np.array(df_test["Distance"]).reshape(len(df_test),1)
    y_train_lm = np.array(10 - df_train["score"])
    lm_model.fit(x_train_lm ,y_train_lm)

    df_test["pred_lm"] = 10 - np.dot(lm_model.coef_,x_test_lm.T)
    RMSE = np.sqrt(np.mean((df_test["score"] - df_test["pred_lm"])**2))
    RMSE_list.append(RMSE)
print("RMSE: ",np.mean(RMSE_list))
RMSE_model_list.append({"model":"Safety Index","RMSE":np.mean(RMSE_list)})

RMSE:  2.927742203640457


In [7]:
# Genetic Programming(López et al., 2018)

$$
\begin{aligned}
y=& 9.153-0.1119 x_6-0.1119 x_7-0.1119 x_8-2.516 e^{-\frac{1.0 x_2}{x_4}} \\
&-0.1119 e^{-1.0 x_1}-0.01742 x_2\left(x_2+e^{-1.0 x_1}+e^{-1.0 x_4}\right) \\
&-0.2239 x_2
\end{aligned}
$$

In [8]:
RMSE_list = []
for i in range(1,11):
    df_train,df_test = import_data(i)
    df_test["pred_gp"] = 9.153 - 0.1119 * df_test["intense_brakes"] - 0.1119 * df_test["sudden_stops"] - 0.1119 * df_test["abrupt_steerings"] - 2.516 * np.exp(-df_test["Average_speed"] / df_test["sudden_starts"]) - \
    0.1119 * np.exp(-df_test["Distance"]) - 0.01742 * df_test["Average_speed"] * (df_test["Average_speed"] + np.exp(-df_test["Distance"]) + np.exp(-df_test["sudden_starts"])) - 0.2239 * df_test["Average_speed"]

    RMSE = np.sqrt(np.mean((df_test["score"] - df_test["pred_gp"])**2))
    RMSE_list.append(RMSE)
print("RMSE: ",np.mean(RMSE_list))
RMSE_model_list.append({"model":"Genetic Programming","RMSE":np.mean(RMSE_list)})

RMSE:  1.2234006634096177


In [9]:
# Bayesian Ridge Regression(Eran et al., 2012)

$$
y = 8.2492 + 0.03783x_1  - 0.2751x_2  - 0.003735x_3  - 0.1322x_4  - 0.01145x_5  - 0.1122 x_6 - 0.1501x_7  -0.1301x_8
$$

In [10]:
from sklearn.linear_model import BayesianRidge
RMSE_list = []
for i in range(1,11):
    df_train,df_test = import_data(i)
    br_model = BayesianRidge(fit_intercept = True,verbose = False)
    x_train_br = np.array(df_train[x_col_names])
    x_test_br = np.array(df_test[x_col_names])
    y_train_br = np.array(df_train["score"])
    br_model.fit(x_train_br,y_train_br)

    df_test["pred_br"] = np.dot(br_model.coef_,x_test_br.T) + br_model.intercept_
    RMSE = np.sqrt(np.mean((df_test["score"] - df_test["pred_br"])**2))
    RMSE_list.append(RMSE)
print("RMSE: ",np.mean(RMSE_list))
RMSE_model_list.append({"model":"BayesianRidge","RMSE":np.mean(RMSE_list)})

RMSE:  1.2789433772391507


In [11]:
# Random Forest(Breiman, 2001)
from sklearn.ensemble import RandomForestRegressor
RMSE_list = []
for i in range(1,11):
    df_train,df_test = import_data(i)
    rf_model = RandomForestRegressor()
    x_train_rf = np.array(df_train[x_col_names])
    x_test_rf = np.array(df_test[x_col_names])
    y_train_rf = np.array(df_train["score"])
    rf_model.fit(x_train_rf,y_train_rf)

    df_test["pred_rf"] = rf_model.predict(x_test_rf)
    RMSE = np.sqrt(np.mean((df_test["score"] - df_test["pred_rf"])**2))
    RMSE_list.append(RMSE)
print("RMSE: ",np.mean(RMSE_list))
RMSE_model_list.append({"model":"Random Forest","RMSE":np.mean(RMSE_list)})

RMSE:  1.2610998923592194


In [12]:
# neural network(Haykin, 2003)
from sklearn.neural_network import MLPRegressor
RMSE_list = []
for i in range(1,11):
    df_train,df_test = import_data(i)
    nn_model = MLPRegressor(max_iter=200,batch_size = 10,learning_rate_init = 0.01)
    x_train_nn = np.array(df_train[x_col_names])
    x_test_nn = np.array(df_test[x_col_names])
    y_train_nn = np.array(df_train["score"])
    nn_model.fit(x_train_nn,y_train_nn)
    
    df_test["pred_nn"] = nn_model.predict(x_test_nn)
    RMSE = np.sqrt(np.mean((df_test["score"] - df_test["pred_nn"])**2))
    RMSE_list.append(RMSE)
print("RMSE: ",np.mean(RMSE_list))
RMSE_model_list.append({"model":"Neral Network","RMSE":np.mean(RMSE_list)})

RMSE:  1.5781743777796506


In [14]:
RMSE_model_comparison = pd.DataFrame(RMSE_model_list)
RMSE_model_comparison = RMSE_model_comparison.sort_values(by = "RMSE")
RMSE_model_comparison

,model,RMSE
1,Genetic Programming,1.223401
3,Random Forest,1.261100
2,BayesianRidge,1.278943
4,Neral Network,1.578174
0,Safety Index,2.927742
